In [1]:
import os
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchnet import meter
from tqdm import tqdm
import numpy as np
import time
import torch.nn.functional as F
import torch.nn as nn


In [2]:
from cqt_loader import CQT
from models import CQTNet 
from config import DefaultConfig
from utility import calc_MAP

# Set Parameters

In [3]:
modelname = 'CQTNet'
load_latest = True
load_model_path = 'check_points/CQTNet.pth'
model_save_path = 'finetuning'

opt = DefaultConfig(load_model_path=load_model_path, notes=model_save_path)

torch.cuda.is_available() False
use_gpu True
cpu


/Users/dirceusilva/.pyenv/versions/3.9.19/envs/audio/lib/python3.9/site-packages/torch/__init__.py:696: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


# step 1: Build DataLoader

In [6]:
train_data = CQT('train', out_length=300)
val_data = CQT('val', out_length=None)
test_data = CQT('test', out_length=None)

train_dataloader = DataLoader(train_data, opt.batch_size, shuffle=True,num_workers=opt.num_workers)
val_dataloader = DataLoader(val_data, 1, shuffle=False, num_workers=1)
test_dataloader = DataLoader(test_data, 1, shuffle=False, num_workers=1)

train -- set_ids: 6191 -- tracks_ids: 40713
val -- set_ids: 2770 -- tracks_ids: 8310
test -- set_ids: 4394 -- tracks_ids: 8788


# step 2: configure model

In [5]:

#opt._parse(kwargs)

model = CQTNet.CQTNet()
model.load_state_dict(torch.load(load_model_path, map_location=opt.device))

print(model)

CQTNet(
  (features): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool3)

In [6]:
model._modules.keys()

odict_keys(['features', 'pool', 'fc0', 'fc1'])

In [7]:
for param in model.parameters():
    param.requires_grad = False

### Change Model

In [8]:
model.fc1 = nn.Linear(300, train_data.get_setids_len())

In [9]:
for param in model.fc1.parameters():
    param.requires_grad = True

In [9]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

features.conv0.weight False
features.norm0.weight False
features.norm0.bias False
features.conv1.weight False
features.norm1.weight False
features.norm1.bias False
features.conv2.weight False
features.norm2.weight False
features.norm2.bias False
features.conv3.weight False
features.norm3.weight False
features.norm3.bias False
features.conv4.weight False
features.norm4.weight False
features.norm4.bias False
features.conv5.weight False
features.norm5.weight False
features.norm5.bias False
features.conv6.weight False
features.norm6.weight False
features.norm6.bias False
features.conv7.weight False
features.norm7.weight False
features.norm7.bias False
features.conv8.weight False
features.norm8.weight False
features.norm8.bias False
features.conv9.weight False
features.norm9.weight False
features.norm9.bias False
fc0.weight False
fc0.bias False
fc1.weight True
fc1.bias True


# To device

In [10]:
model.to(opt.device)
print(model)

CQTNet(
  (features): Sequential(
    (conv0): Conv2d(1, 32, kernel_size=(12, 3), stride=(1, 1), padding=(6, 0), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(32, 64, kernel_size=(13, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=(0, 1), dilation=1, ceil_mode=False)
    (conv2): Conv2d(64, 64, kernel_size=(13, 3), stride=(1, 1), bias=False)
    (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), dilation=(1, 2), bias=False)
    (norm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (pool3)

#  step 3: criterion and optimizer

In [11]:
criterion = torch.nn.CrossEntropyLoss()
lr = opt.lr
if opt.parallel is True:
    optimizer = torch.optim.Adam(model.module.parameters(), lr=lr, weight_decay=opt.weight_decay)
else:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=opt.weight_decay)
    
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=opt.lr_decay, patience=2, verbose=True, min_lr=5e-6)

AttributeError: 'CQTNet' object has no attribute 'module'

# step 4: Train

### Validation Function

In [ ]:
@torch.no_grad()
def val_slow(model, dataloader, epoch):
    
    model.eval()
    total, correct = 0, 0
    labels, features = None, None

    for ii, (data, label) in enumerate(dataloader):
        input_data = data.to(opt.device)
        #print(input.shape)
        score, feature = model(input_data)
        feature = feature.data.cpu().numpy()
        label = label.data.cpu().numpy()
        if features is not None:
            features = np.concatenate((features, feature), axis=0)
            labels = np.concatenate((labels,label))
        else:
            features = feature
            labels = label
    features = norm(features)

    dis2d = -np.matmul(features, features.T) # [-1,1] Because normalized, so mutmul is equal to ED
    np.save('dis.npy',dis2d)
    np.save('label.npy',labels)

    MAP, top10, rank1 = calc_MAP(dis2d, labels)

    print(epoch, MAP, top10, rank1 )
    
    model.train()
    
    return MAP

### Training

In [ ]:
best_MAP=0
    
val_slow(model, val_dataloader, -1)
for epoch in range(opt.max_epoch):
    running_loss = 0
    num = 0
    for data, label in tqdm(train_dataloader):
        
        # train model
        input_data = data.requires_grad_()
        input_data = input_data.to(opt.device)
        
        target = label.to(opt.device)

        optimizer.zero_grad()
        score, _ = model(input_data)
        loss = criterion(score, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num += target.shape[0]
        
    running_loss /= num 
    
    print(running_loss)
    
    if opt.parallel is True:
        model.module.save(opt.notes)
    else:
        model.save(opt.notes)
    
    # update learning rate
    scheduler.step(running_loss) 
    
    # validate
    MAP=0
    MAP += val_slow(model, val_dataloader, epoch)
            
    if MAP>best_MAP:
        best_MAP=MAP
        print('*****************BEST*****************')
    print('')
    model.train()

# step 5: Test

In [ ]:
def test(**kwargs):
    opt.batch_size=1
    opt.num_workers=1
    opt.model = 'CQTNet'
    opt.load_latest = False
    opt.load_model_path = 'check_points/CQTNet.pth'
    opt._parse(kwargs)
    
    model = getattr(models, opt.model)() 
    #print(model)
    if opt.load_latest is True:
        model.load_latest(opt.notes)
    elif opt.load_model_path:
        model.load(opt.load_model_path)
    model.to(opt.device)

    val_data = CQT('val', out_length=None)
    test_data = CQT('test', out_length=None)
    val_dataloader = DataLoader(val_data, 1, shuffle=False,num_workers=1)
    test_dataloader = DataLoader(test_data, 1, shuffle=False,num_workers=1)
    
    val_slow(model, val_dataloader, 0)